# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [4]:
# your code here
import pandas as pd

data = pd.read_csv('/tic-tac-toe.csv')
# Inspect the dataset
print(data.head())
print(data.info())
print(data.describe())

# Check the distribution of the target variable
print(data['class'].value_counts())

# Convert categorical values to numeric
data = data.applymap(lambda x: 1 if x == 'x' else (-1 if x == 'o' else 0))
data['class'] = pd.to_numeric(data['class'])

# Separate the inputs and output
predictors = data.drop(columns=['class'])
target = data['class']

# Normalize the input data.
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()


  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
         TL   TM   TR   ML   MM   MR   BL   BM   BR class
count   958  958  958  958  958  958  958  958  958   958
unique    3    3    3    3    3    3    3    3    3     2
top       x    x  

<ipython-input-4-7a12daa73632>:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: 1 if x == 'x' else (-1 if x == 'o' else 0))


,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1.034629,1.106252,1.034629,1.106252,-1.241346,-1.222956,1.034629,-1.222956,-1.230913
1,1.034629,1.106252,1.034629,1.106252,-1.241346,-1.222956,-1.230913,1.106252,-1.230913
2,1.034629,1.106252,1.034629,1.106252,-1.241346,-1.222956,-1.230913,-1.222956,1.034629
3,1.034629,1.106252,1.034629,1.106252,-1.241346,-1.222956,-1.230913,-0.058352,-0.098142
4,1.034629,1.106252,1.034629,1.106252,-1.241346,-1.222956,-0.098142,-1.222956,-0.098142


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [12]:
# your code here

import tensorflow
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

# Create the model

model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(predictors_norm.shape[1],)))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model with the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Save the model using the .keras extension
model.save('tic-tac-toe.keras')

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3208 - loss: 1.2365 - val_accuracy: 0.9627 - val_loss: 0.6302
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9878 - loss: 0.4614 - val_accuracy: 1.0000 - val_loss: 0.2013
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.1375 - val_accuracy: 1.0000 - val_loss: 0.0620
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0420 - val_accuracy: 1.0000 - val_loss: 0.0250
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0185 - val_accuracy: 1.0000 - val_loss: 0.0136
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0113 - val_accuracy: 1.0000 - val_loss: 0.0087
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0075 - val_accuracy: 1.0000 - val_loss: 0.0062
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 1.0000 - val_loss: 0.0047
Ep

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [14]:
# your code here

import numpy as np

# Load the saved model
loaded_model = keras.models.load_model('tic-tac-toe.keras')

# Select a few random rows from the test dataset
random_indices = np.random.choice(X_test.shape[0], size=5, replace=False)
random_samples = X_test.iloc[random_indices]  # Use .iloc for integer-based indexing
random_labels = y_test.iloc[random_indices]  # Use .iloc for integer-based indexing

# Make predictions on the selected rows
predictions = loaded_model.predict(random_samples)
predicted_classes = np.argmax(predictions, axis=1)

# Print the predictions and the actual labels
for i in range(len(random_samples)):
    print(f"Sample {i+1}:")
    print(f"Predicted class: {predicted_classes[i]}")
    print(f"Actual class: {random_labels.iloc[i]}")  # Use .iloc to access the value
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Sample 1:
Predicted class: 0
Actual class: 0

Sample 2:
Predicted class: 0
Actual class: 0

Sample 3:
Predicted class: 0
Actual class: 0

Sample 4:
Predicted class: 0
Actual class: 0

Sample 5:
Predicted class: 0
Actual class: 0



## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [16]:
# your code here
import tensorflow as tf

# Define a custom learning rate
learning_rate = 0.0001

# Create a custom Adam optimizer with the specified learning rate
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model with the custom optimizer
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the training data
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model with the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 6.7670e-08 - val_accuracy: 1.0000 - val_loss: 1.0762e-07
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 5.6686e-08 - val_accuracy: 1.0000 - val_loss: 1.0514e-07
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 6.2757e-08 - val_accuracy: 1.0000 - val_loss: 9.8927e-08
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 5.3781e-08 - val_accuracy: 1.0000 - val_loss: 9.2718e-08
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 5.1727e-08 - val_accuracy: 1.0000 - val_loss: 8.7337e-08
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 6.2266e-08 - val_accuracy: 1.0000 - val_loss: 7.9473e-08
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 5.2583e-08 - val_accuracy: 1.0000 - val_loss: 7.3678e-08
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy:

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here